In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
path = 'csv_files/' #path of IHDP files, downloaded from https://www.fredjo.com/
def processed_data(i):
    train_data = pd.read_csv(path+'1000_train'+str(i)+'.csv').values
    train_data[:,13] = train_data[:,13]-1 #processing according to CEVAE
    test_data = pd.read_csv(path+'1000_test'+str(i)+'.csv').values
    test_data[:,13] = test_data[:,13]-1 #processing according to CEVAE
    train_label = np.zeros((len(train_data),3))
    t= pd.read_csv(path+'1000_train_t'+str(i)+'.csv').values.flatten()
    yf = pd.read_csv(path+'1000_train_yf'+str(i)+'.csv').values.flatten()
    ycf = pd.read_csv(path+'1000_train_ycf'+str(i)+'.csv').values.flatten()
    ite_test = pd.read_csv(path+'1000_test_ite'+str(i)+'.csv').values
    return train_data,test_data,t,yf,ite_test
    

In [5]:
import pickle
import numpy as np
from tqdm import tqdm
from tqdm.auto import trange
from catenets.models.jax import TNet, SNet
from sklearn.metrics import mean_squared_error 
t_rmse = []
s_rmse = []
t = TNet()
s = SNet(penalty_orthogonal=0.01)
for i in trange(1000, position=0, desc="replication", leave=True, colour='black',):
    train_data,test_data,w,y,ite_test = processed_data(i)  
    t.fit(train_data, y, w)
    ite_preds = t.predict(test_data, return_po=False)  # predict potential outcomes too

    t_rmse.append(mean_squared_error(ite_preds, ite_test)**0.5)
    s.fit(train_data, y, w)
    ite_preds = s.predict(test_data, return_po=False)  # predict potential outcomes too

    s_rmse.append(mean_squared_error(ite_preds, ite_test)**0.5)

replication:   0%|          | 0/1000 [00:00<?, ?it/s]

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [6]:
#print the results
from scipy.stats import sem
results = np.array(t_rmse)
print(
    '\n The average rpehe of test data are: ',
    np.mean(results),
    'The standard error of epehe in test data are: ',
    sem(results))


 The average rpehe of test data are:  1.769540510248449 The standard error of epehe in test data are:  0.05425740595199925


In [7]:
#print the results
results = np.array(s_rmse)
print(
    '\n The average rpehe of test data are: ',
    np.mean(results),
    'The standard error of epehe in test data are: ',
    sem(results))


 The average rpehe of test data are:  1.2871888543948293 The standard error of epehe in test data are:  0.03902574670749591
